In [33]:
# TODO
# catologue risks
# refer to hse (health and safety executive)
# We have caught carp, barbel, roach, rudd, perch, skimmer, bream, gudgeon, tench,
#

In [34]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [35]:
from typing import Literal
from pydantic import BaseModel, Field


class Risk(BaseModel):
    """Describes a risk."""

    hazard: str = Field(
        description=(
            "Concise description of the hazard. "
            "A hazard is something that may cause harm or damage."
        )
    )
    risks: list[str] = Field(
        description=(
            "List of risks associated with the hazard. "
            "A risk is the harm that may occur from the hazard. "
            "Describe each risk in a sentence."
        )
    )
    affects: Literal["Young people", "Leaders", "All present"] = Field(
        description="Who is at risk. Leaders are adult Scout leaders."
    )


class Risks(BaseModel):
    """List of risks."""

    risks: list[Risk]

In [36]:
controls = """
Controls are ways of making the activity safer by removing or reducing the risk.  
For example, you may use a different piece of equipment or you might change the way you do the activity.
"""

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

# TODO
# How to consider children's behaviour
# Correctly classify who is at risk - may need a reviewer
# Inject examples for multi-shot
# How to describe each value of a literal in pydantic

system = """
A scout leader planning an activity for Scouts (aged 10-14) needs to perform a 
risk assessment for an activity at a given location.

Your task is to perform a risk assessment identifying hazards and the risks
associated with them.

When identifying hazards and risks, pay attention to the location as this can 
introduce additional hazards.

Always consider the impact that poor behaviour of young people could have.

Use plain British english and commonly used terms in your response.
"""

prompt = """
Location: {location}

Activity: {activity}

Identify {n} risks. Rank them from highest impact to lowest impact.
"""

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        HumanMessagePromptTemplate.from_template(prompt),
    ]
)

In [38]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=1.0)
llm = model.with_structured_output(Risks, include_raw=True)
app = chat_template | llm

risks = app.invoke(
    {
        # "activity": "Pumpkin carving with tee light candles. Pumpkin carving will be done on tables in groups of 2, 2 groups per table.",
        # "activity": "Variety of indoor games such as dodgeball, crab football etc.",
        "activity": "Scouts making pancakes in the kitchen when being supervised by 2 adults.",
        "location": "The scout kitchen is a medium sized kitchen similar to a typical family kitchen",
        # "location": "The scout hut is large brick building with a hard wooden floor and a raised stage at the front. I contains a number of pillars to support the roof and stacked chairs.",
        "n": 5,
    }
)

# TODO would generally run this generation several times and filter

In [39]:
print(risks["parsed"].model_dump_json(indent=3))

{
   "risks": [
      {
         "hazard": "Hot surfaces and equipment (stove, grill)",
         "risks": [
            "Burns from touching hot surfaces or equipment.",
            "Fires caused by spills and overheating."
         ],
         "affects": "All present"
      },
      {
         "hazard": "Sharp utensils (knives, spatulas)",
         "risks": [
            "Cuts from using knives or other sharp tools.",
            "Injury from improper handling of utensils."
         ],
         "affects": "Young people"
      },
      {
         "hazard": "Slips and falls due to spills",
         "risks": [
            "Injuries from slipping on spilled batter or oil.",
            "Falls causing bruises or sprains."
         ],
         "affects": "All present"
      },
      {
         "hazard": "Food allergies and intolerances",
         "risks": [
            "Health issues or severe allergic reactions.",
            "Injury from consuming inappropriate ingredients."
         ],
 

In [40]:
# FLOW
# identify risks -> generate x3 -> combine similar risks